## Connection to ElasticSearch

In [5]:
from pprint import pprint
from elasticsearch import Elasticsearch
from elastic_transport import TransportError

In [6]:
from elasticsearch import Elasticsearch

es = Elasticsearch(
    "http://localhost:9200",
    headers={
        "Content-Type": "application/json",
        "Accept": "application/json"
    }
)

print(es.ping())

True


In [8]:
client_info = es.info()
pprint(client_info.body)

{'cluster_name': 'docker-cluster',
 'cluster_uuid': 'kdgaRXuGS8yh7rxokb9Jwg',
 'name': 'eb3cb2c67bb9',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2024-08-05T10:05:34.233336849Z',
             'build_flavor': 'default',
             'build_hash': '1a77947f34deddb41af25e6f0ddb8e830159c179',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '9.11.1',
             'minimum_index_compatibility_version': '7.0.0',
             'minimum_wire_compatibility_version': '7.17.0',
             'number': '8.15.0'}}


## Create index

In [14]:
es.indices.delete(index='my_index', ignore_unavailable=True)
es.indices.create(index='my_index', settings={
    'index': {'number_of_shards': 1,
              'number_of_replicas': 1,}
})

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index'})